
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%run ../Includes/Copy-Datasets

## 📦 Parsing JSON Data

Spark SQL provides robust tools to work with **nested JSON structures** often found in real-world datasets:

- JSON fields can be parsed into structured formats (e.g., structs) for easier access.
- Extracting and flattening these nested fields helps make the data usable for querying and transformations.
- Deriving a schema beforehand ensures accurate parsing and efficient performance.

In [0]:
SELECT * FROM customers;

In [0]:
--Profile field will show as string
DESCRIBE customers;

In [0]:
--Use the colon syntax to traverse nested data structures
SELECT customer_id, profile:first_name, profile:address:country 
FROM customers;

## 🧱 Understanding Struct Types in Spark

A **struct** is a complex data type that groups multiple fields together into a single column.

- Think of it like a mini table or a nested object.
- It allows storing multiple related values (e.g., name, age, address) in one column.
- Structs are commonly seen when working with JSON, where data is nested.

### 🔹 Why use struct types?

- Helps keep related attributes grouped together.
- Makes it easier to work with **nested data** (like JSON or hierarchical data).
- Allows for organized data modeling in a single column.

### 🔹 Accessing fields

You can access fields inside a struct using dot notation:

In [0]:
--This will fail because this function requires the schema of the JSON object
SELECT from_json(profile) AS profile_struct
  FROM customers;

In [0]:
--Gather the sample data of our JSON value with non null fields
SELECT profile 
FROM customers 
LIMIT 1;

In [0]:
--Copy the Sample data and provide it to the schmea_of_json function and store these parsed recrods in a temporary view
CREATE OR REPLACE TEMP VIEW parsed_customers AS
  SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Thomas","last_name":"Lane","gender":"Male","address":{"street":"06 Boulevard Victor Hugo","city":"Paris","country":"France"}}')) AS profile_struct
  FROM customers;
  
SELECT * FROM parsed_customers;

In [0]:
--The new column will have a struct data type
DESCRIBE parsed_customers;

In [0]:
--With struct type, we can interact with the subfields using standard period or dot syntax instead of column syntax
SELECT customer_id, profile_struct.first_name, profile_struct.address.country
FROM parsed_customers;

In [0]:
--Once a JSON string is converted to a struct type, use the star operation to flatten fields into columns
CREATE OR REPLACE TEMP VIEW customers_final AS
  SELECT customer_id, profile_struct.*
  FROM parsed_customers;
  
SELECT * FROM customers_final;

In [0]:
SELECT order_id, customer_id, books
FROM orders;

## 💥 Explode Function

The `explode` function is used to transform **arrays into multiple rows**.

- This operation is essential when working with columns that contain array-type data.
- It helps to break down and analyze each element of an array as a separate row.

In [0]:
SELECT order_id, customer_id, explode(books) AS book 
FROM orders;

## 🧺 Collecting Rows

To aggregate values from a group of rows:

- Use functions like `collect_set` or `collect_list` to **combine values into arrays**.
- This is useful for summarizing data while preserving uniqueness or order.

In [0]:
SELECT customer_id,
  collect_set(order_id) AS orders_set,
  collect_set(books.book_id) AS books_set
FROM orders
GROUP BY customer_id;

## 🪄 Flattening Arrays and Structs

Flattening nested structures makes data more accessible and easier to analyze:

- Structs can be **expanded into individual columns**.
- Arrays can be exploded and then flattened using additional transformations.

This process turns deeply nested data into a tabular format that is easier to query and work with.


In [0]:
SELECT customer_id,
  collect_set(books.book_id) As before_flatten,
  array_distinct(flatten(collect_set(books.book_id))) AS after_flatten
FROM orders
GROUP BY customer_id;

## 🔗 Join Operations

Spark SQL supports a variety of **join types** to combine datasets:

- **Inner Join**
- **Left / Right Outer Join**
- **Full Outer Join**
- **Left Semi / Anti Join**
- **Cross Join**

These joins are used to enrich datasets by matching and merging records from different sources based on common keys.


In [0]:
CREATE OR REPLACE VIEW orders_enriched AS
SELECT *
FROM (
  SELECT *, explode(books) AS book 
  FROM orders) o
INNER JOIN books b
ON o.book.book_id = b.book_id;

SELECT * FROM orders_enriched;

## 🔁 Set Operations

Set-based operations allow you to compare and combine datasets efficiently:

- **UNION** combines datasets by rows.
- **INTERSECT** returns only common records.
- **EXCEPT/MINUS** subtracts one dataset from another.

These operations are helpful for dataset comparison, deduplication, and validation.


In [0]:
CREATE OR REPLACE TEMP VIEW orders_updates
AS SELECT * FROM parquet.`${dataset.bookstore}/orders-new`;

SELECT * FROM orders 
UNION 
SELECT * FROM orders_updates;

In [0]:
SELECT * FROM orders 
INTERSECT 
SELECT * FROM orders_updates;

In [0]:
SELECT * FROM orders 
MINUS 
SELECT * FROM orders_updates;

## 🔄 Reshaping Data with Pivot

The **PIVOT** clause allows you to **transform rows into columns**, enabling powerful data reshaping:

- Ideal for aggregation across categories.
- Frequently used in reporting, dashboarding, and preparing features for machine learning.

Pivoting simplifies comparative analysis and enables better data visualization.

In [0]:
CREATE OR REPLACE TABLE transactions AS

SELECT * FROM (
  SELECT
    customer_id,
    book.book_id AS book_id,
    book.quantity AS quantity
  FROM orders_enriched
) PIVOT (
  sum(quantity) FOR book_id in (
    'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
    'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
  )
);

SELECT * FROM transactions;